In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.datasets import load_digits

In [3]:
digits = load_digits()
x = digits.data
y = digits.target
n_samples, n_features = x.shape

In [5]:
batch_size = 500
n_batches = n_samples // batch_size

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
x[0]

array([ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
       15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
       12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
        0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
       10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.])

In [8]:
latent_dim = 2
n_hidden = 64


In [11]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def forward_vae(x):
    input_size = x.shape[0]
    hidden_size = 4
    latent_size = 2

    # Encoder weights
    W1 = np.random.randn(hidden_size, input_size)
    b1 = np.zeros(hidden_size)

    # Mean and logvar layer weights
    W_mu = np.random.randn(latent_size, hidden_size)
    b_mu = np.zeros(latent_size)

    W_logvar = np.random.randn(latent_size, hidden_size)
    b_logvar = np.zeros(latent_size)

    # Decoder weights
    W3 = np.random.randn(hidden_size, latent_size)
    b3 = np.zeros(hidden_size)

    W4 = np.random.randn(input_size, hidden_size)
    b4 = np.zeros(input_size)

    # ---- Encoder ----
    h1 = relu(np.dot(W1, x) + b1)

    mu = np.dot(W_mu, h1) + b_mu
    logvar = np.dot(W_logvar, h1) + b_logvar

    # ---- Reparameterization Trick ----
    std = np.exp(0.5 * logvar)
    epsilon = np.random.randn(*std.shape)
    z = mu + epsilon * std

    # ---- Decoder ----
    h3 = relu(np.dot(W3, z) + b3)
    x_hat = sigmoid(np.dot(W4, h3) + b4)

    return x_hat, mu, logvar, z

In [12]:
x = np.random.rand(6)  # input with 6 features
x_hat, mu, logvar, z = forward_vae(x)

print("Reconstructed x:", x_hat)
print("Latent vector z:", z)

Reconstructed x: [1.00000000e+00 1.00000000e+00 1.00000000e+00 3.33688671e-40
 2.16755943e-12 4.81543665e-19]
Latent vector z: [-37.94585251   0.08137975]


In [15]:
def vae_forward(x):
    # Encoder weights (3 input → 2 hidden)
    W_enc = np.array([[0.1, 0.2, 0.3],
                      [0.4, 0.5, 0.6]])
    b_enc = np.array([0.0, 0.0])
    
    # Separate weights for mean and logvar (2 hidden → 2 latent)
    W_mu = np.array([[0.2, 0.1],
                     [0.3, 0.4]])
    b_mu = np.array([0.1, -0.1])

    W_logvar = np.array([[0.1, -0.2],
                         [0.05, 0.3]])
    b_logvar = np.array([-0.2, 0.2])

    # Decoder (2 latent → 3 output)
    W_dec = np.array([[0.1, 0.2],
                      [0.3, 0.4],
                      [0.5, 0.6]])
    b_dec = np.array([0.0, 0.0, 0.0])
    
    # Encoder forward pass
    h = relu(np.dot(W_enc, x) + b_enc)   # (2,)
    
    mu = np.dot(W_mu, h) + b_mu          # (2,)
    logvar = np.dot(W_logvar, h) + b_logvar  # (2,)

    std = np.exp(0.5 * logvar)
    epsilon = np.random.randn(*std.shape)
    z = mu + std * epsilon               # Reparameterization

    x_hat = sigmoid(np.dot(W_dec, z) + b_dec)

    return x_hat, z, mu, logvar

In [16]:
x = np.array([0.5, 0.2, 0.9])
x_hat, z, mu, logvar = vae_forward(x)

print("Input x:", x)
print("Mean (μ):", mu)
print("Logvar:", logvar)
print("Sampled z:", z)
print("Reconstructed x_hat:", x_hat)

Input x: [0.5 0.2 0.9]
Mean (μ): [0.256 0.344]
Logvar: [-0.332  0.47 ]
Sampled z: [1.10179176 0.22071844]
Reconstructed x_hat: [0.53850433 0.60320203 0.66448083]
